In [1]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH02-Runnable")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Runnable


# RunnablePassthrough

- 입력을 변경하지 않고 그대로 전달하거나, 추가 키를 더하여 전달 
- RunnablePassthrough : 단독 호출하면 입력을 그대로 전달 
- RunnablePassthrough.assign() : 입력 받아 assign 함수에 추가 인자 더함 

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [4]:
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 5),
    modified=lambda x: x["num"] + 10,
)

In [5]:
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 5}, 'modified': 11}

In [6]:
runnable_ = RunnablePassthrough.assign(mult=lambda x: x["num"] * 5)
runnable_.invoke({"num": 1})

{'num': 1, 'mult': 5}

- passed : RunnablePassthrough 만 호출. 입력받은 'num'값만 전달
- extra : RunnablePassthrough로 입력받은 'num'값에 대하여 추가된 assign() 함수 처리
- modified : 입력된 'num' 값으로 사용자함수(lambda) 계산

# 검색기 예제

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [ ]:
# Vector Store 생성
vector_store = FAISS.from_texts(
    [
        "영희는 스칼라 주식회사에서 근무하고 있습니다.",
        "철수는 같은 회사에서 근무 하였습니다.",
        "영희 직업은 개발자 입니다.",
        "철수의 직업은 컨텐츠 기획자 입니다.",
    ],
    embedding=OpenAIEmbeddings(),
)

# Vector Store를 검색기로 사용
retriever = vector_store.as_retriever()

# 템플릿 정의
template = """
Answer the question based only on the following context: {context}
Question: {question}
"""

# Prompt 정의
prompt = PromptTemplate.from_template(template)

# 모델 정의/초기화
model = ChatOpenAI(model_name="gpt-4o-mini")


# 문서 포맷 함수
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])


# Chain 구성
retrieval_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [9]:
retrieval_chain.invoke("영희의 직업은 무엇인가요?")

'영희의 직업은 개발자입니다.'

In [10]:
retrieval_chain.invoke("영희는 현재 어디에서 근무하나요?")

'영희는 스칼라 주식회사에서 근무하고 있습니다.'

-----
* End of Document *